| Step                                           | Task                                     | Concrete actions (code-level)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Recommended packages                                        | Rationale                                                                                                                          |
| ---------------------------------------------- | ---------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------- |
| **7 .1  Baseline Linear Quantile Regression**  | *Parametric benchmark*                   | 1. Build a `sklearn.pipeline.Pipeline` that<br> • median-imputes **within each CV fold**<br> • standardises numeric features (robust scaler).<br>2. Fit three `statsmodels.QuantReg` models (τ = 0.05, 0.50, 0.95).<br>3. Store coefficients, pinball-loss series and residuals.                                                                                                                                                                                                                                                                               | `pandas`, `statsmodels`                                     | Linear QR is transparent, fast and gives a sanity check that later ML models really add value.                                     |
| **7 .2  LightGBM multi-quantile GBM**          | *Strong non-linear baseline*             | 1. For each quantile in **{0.05, 0.25, 0.50, 0.75, 0.95}** create a clone of `lightgbm.LGBMRegressor(objective="quantile", alpha=τ, n_estimators=500, learning_rate=0.05, max_depth=-1, min_child_samples=30)`.  <br>2. Wrap the five models in `sklearn.multioutput.MultiOutputRegressor` so they train in parallel per fold. <br>3. After each fold, feed `(X_cal, y_cal)` to `mapie.regression.MapieQuantileRegressor` (`method="quantile", cv="prefit"`) with `alpha=[0.1,0.5,0.9]` to *conformalise* the raw bounds and guarantee finite-sample coverage. | `lightgbm`, `mapie`, `scikit-learn`                         | Gradient boosting usually wins pure accuracy but is prone to under-coverage; CQR (split-conformal) fixes that without re-training. |
| **7 .3  Quantile Regression Forest (QRF)**     | *Non-parametric full-distribution model* | 1. Initialise one `quantile_forest.RandomForestQuantileRegressor(n_estimators=600, max_depth=None, min_samples_leaf=5, sample_fraction=0.9, random_state=42)`.<br>2. After fitting on the train window, predict `quantiles=[0.05,0.25,0.50,0.75,0.95]` in one call.<br>3. Optionally feed predictions into MAPIE-CQR exactly as for LightGBM if empirical coverage < nominal.<br>4. Log leaf-level distributions for SHAP later (`tree_explainer = shap.TreeExplainer(model)`).                                                                                | `quantile-forest` (Zillow implementation), `MAPIE`, `shap`  | QRF natively returns *any* quantile without extra models, handles heavy tails, and remains fast thanks to Cython optimisation.     |
| **7 .4  Optional residual-bootstrap ensemble** | *Distribution-free fallback*             | 1. Train a point-forecast GBM (`objective="mse"`).<br>2. Collect rolling residuals `e_t`.<br>3. Generate **B = 1 000** bootstrap paths per forecast: `ŷ_t + e_b` where `e_b` is block-bootstrapped (stationary bootstrap) to respect autocorrelation.<br>4. Use empirical 5ᵗʰ/95ᵗʰ percentiles as interval bounds.                                                                                                                                                                                                                                             | `lightgbm`, `arch.bootstrap.StationaryBootstrap`            | Simple, assumption-light; good stress-test if quantile models misfire.                                                             |


Package Selection Cheat Sheet:

| Library                   | Why use it here?                                                                          | Caveats                                                                           |
| ------------------------- | ----------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------- |
| **`quantile-forest`**     | One model → many quantiles; Cython-fast; integrates with SHAP.                            | Slightly higher RAM usage than GBR.                                               |
| **`lightgbm` (quantile)** | High predictive power; GPU option; accepts categorical splits directly.                   | Raw intervals under-cover → always wrap in MAPIE.                                 |
| **`MAPIE` / CQR**         | Drop-in conformal calibration with finite-sample guarantees, supports time-series split.  | Needs a held-out calibration slice; adds a tiny constant δ so track final widths. |
| **`statsmodels`**         | Mature linear QR, robust SEs for coefficient tables.                                      | No native categorical handling → encode beforehand.                               |
| **`arch.bootstrap`**      | Stationary bootstrap suited to overlapping residuals.                                     | Computationally heavier than split-conformal.                                     |
| **`shap`**                | Consistent TreeSHAP explanations for feature importance across τ.                         | Large memory on huge forests; sample for plots.                                   |
